In [1]:
using TaylorModels, IntervalArithmetic, TaylorSeries

In [2]:
const ∫ = integrate
const Interval = IntervalArithmetic.Interval

import Base:^

^(x::Interval{Float64}, n::Int) = pow(x, n)

^ (generic function with 92 methods)

In [17]:
n = 12  # order
a, b = set_variables("a b", order=n)

2-element Array{TaylorSeries.TaylorN{Float64},1}:
  1.0 a + 𝒪(‖x‖¹³)
  1.0 b + 𝒪(‖x‖¹³)

In [4]:
function calculate_set(U::TaylorN, V::TaylorN, bounds)

    xs = []
    ys = []

    # iterate over 4 sides of initial box

    num_points = 50

    b = bounds[2].lo
    for a in linspace(bounds[1].lo, bounds[2].hi, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    a = bounds[1].hi
    for b in linspace(bounds[2].lo, bounds[2].hi, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    b = bounds[2].hi
    for a in linspace(bounds[2].hi, bounds[2].lo, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    a = bounds[1].lo
    for b in linspace(bounds[2].hi, bounds[2].lo, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    return mid.(xs), mid.(ys)

end

calculate_set (generic function with 1 method)

In [5]:
function integ(u0, v0, t_interval, bounds)
    
    u = u0
    v = v0
    u_new = u0   # so exist outside loop
    v_new = v0

    # build up Taylor series by Picard:
    for i in 1:n+1   # how many iterations are required?
        u_new = u0 + ∫(   2  * u * (1 - v) )
        v_new = v0 + ∫(       -v * (1 - u) )

        u, v = u_new, v_new
    end

    # prepare Taylor Model:
    uu = TaylorModel(n, Interval(t_interval.lo), t_interval, u, 0..0, bounds)
    vv = TaylorModel(n, Interval(t_interval.lo), t_interval, v, 0..0, bounds)

    uu_new = ∫( 2 * uu * (1 - vv), u0[0] )
    vv_new = ∫(    -vv * (1 - uu), v0[0] )
    
    uΔ = uu_new.Δ
    vΔ = vv_new.Δ
    
    # make sure the intervals contain 0:
    m = mag(uΔ)
    uΔ = -m..m
    
    m = mag(vΔ)
    vΔ = -m..m
    
    @show uΔ, vΔ

     while ! ((uu_new.Δ ⊆ uu.Δ) && (vv_new.Δ ⊆ vv.Δ))
        uΔ *= 2
        vΔ *= 2
        
        @show uΔ, vΔ
        
        uu = TaylorModel(n, Interval(t_interval.lo), t_interval, u, uΔ, bounds)
        vv = TaylorModel(n, Interval(t_interval.lo), t_interval, v, vΔ, bounds)
        
        uu_new = ∫( 2 * uu * (1 - vv), u0[0] )
        vv_new = ∫(    -vv * (1 - uu), v0[0] )
        
    end
        

    # only need to bound the integral, not actually carry out
    # the whole integral

    # contract:

    for i in 1:10
       uu, vv = uu_new, vv_new
       uu_new = ∫( 2 * uu * (1 - vv), u0[0] )
       vv_new = ∫(    -vv * (1 - uu), v0[0] )
    end

    U = uu_new(t_interval.hi)
    V = vv_new(t_interval.hi)
    
    return U, V, uu_new, vv_new
end

integ (generic function with 1 method)

In [6]:
using Plots
gr()

Plots.GRBackend()

In [7]:
time_step = 0.05

h = 0.05
bounds = (-h..h) * ones(2)  # bounds on a and b

u0 = Taylor1([(1..1) + a], n)   # initial conditions as function of a, b
v0 = Taylor1([(3..3) + b], n)   

@time U, V, uu_new, vv_new = integ(u0, v0, 0..time_step, bounds);

(uΔ, vΔ) = ([-3.4631e-13, 3.4631e-13], [-2.03686e-13, 2.03686e-13])
(uΔ, vΔ) = ([-6.9262e-13, 6.9262e-13], [-4.07372e-13, 4.07372e-13])
  1.556512 seconds (3.60 M allocations: 270.070 MiB, 4.43% gc time)


In [8]:
uu_new

TaylorModels.TaylorModel{Float64,TaylorSeries.TaylorN{IntervalArithmetic.Interval{Float64}}}(12, [0, 0], [0, 0.0500001],   [1, 1] + [1, 1] a + 𝒪(‖x‖¹³) + ( [-4, -4] + [-4, -4] a + [-2, -2] b + [-2, -2] a b + 𝒪(‖x‖¹³)) t + ( [8, 8] + [5, 5] a + [8, 8] b + [-3, -3] a² + [7, 7] a b + [2, 2] b² + [-1, -1] a² b + [2, 2] a b² + 𝒪(‖x‖¹³)) t² + ( [-6.66667, -6.66666] + [9.33333, 9.33334] a + [-12.6667, -12.6666] b + [14.9999, 15.0001] a² + [0.666666, 0.666667] a b + [-7.33334, -7.33333] b² + [-1.00001, -0.999999] a³ + [12.9999, 13.0001] a² b + [-4.66667, -4.66666] a b² + [-1.33334, -1.33333] b³ + [-0.333334, -0.333333] a³ b + [2.66666, 2.66667] a² b² + [-1.33334, -1.33333] a b³ + 𝒪(‖x‖¹³)) t³ + ( [-9.33334, -9.33333] + [-48.8334, -48.8333] a + [-5.33334, -5.33333] b + [-26.5001, -26.4999] a² + [-60.5001, -60.4999] a b + [4.33333, 4.33334] b² + [12.7499, 12.7501] a³ + [-45.3334, -45.3333] a² b + [-20.6667, -20.6666] a b² + [3.66666, 3.66667] b³ + [-0.250001, -0.249999] a⁴ + [9.74999, 9.75001] a

In [ ]:
xs, ys = calculate_set(U, V, bounds)

In [9]:
plot(xs, ys, aspect_ratio=1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.78 
 
 
 0.80 
 
 
 0.82 
 
 
 0.84 
 
 
 0.86 
 
 
 2.950 
 
 
 2.975 
 
 
 3.000 
 
 
 3.025 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [10]:
u0 = Taylor1([U], n)   # initial condition as function of a, b
v0 = Taylor1([V], n)   # initial condition as function of a, b

U, V = integ(u0, v0, time_step..(2*time_step), bounds);

xs, ys = calculate_set(U, V, bounds)

(uΔ, vΔ) = ([-2.90023e-13, 2.90023e-13], [-1.62666e-13, 1.62666e-13])
(uΔ, vΔ) = ([-5.80046e-13, 5.80046e-13], [-3.25332e-13, 3.25332e-13])


([0.646264, 0.647618, 0.648972, 0.650326, 0.65168, 0.653034, 0.654388, 0.655742, 0.657095, 0.658448  …  0.643931, 0.64419, 0.644448, 0.644707, 0.644967, 0.645226, 0.645485, 0.645744, 0.646004, 0.646264], [2.888, 2.88848, 2.88897, 2.88945, 2.88994, 2.89043, 2.89091, 2.8914, 2.89188, 2.89237  …  2.90559, 2.90364, 2.90168, 2.89973, 2.89777, 2.89582, 2.89386, 2.89191, 2.88995, 2.888])

In [11]:
plot(xs, ys, aspect_ratio=1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.64 
 
 
 0.66 
 
 
 0.68 
 
 
 0.70 
 
 
 2.900 
 
 
 2.925 
 
 
 2.950 
 
 
 2.975 
 
 
 3.000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [18]:
step = 0.1

h = 0.05
bounds = (-h..h) * ones(2)  # bounds on a and b

u0 = Taylor1([(1..1) + (1..1)*a], n)   # initial condition as function of a, b
v0 = Taylor1([(3..3) + (1..1)*b], n)   # initial condition as function of a, b

U, V = integ(u0, v0, 0..step, bounds)

xs, ys = calculate_set(U, V, bounds)

p = plot(xs, ys, aspect_ratio = 1, leg=false)

for i in 1:60
    @show i
    u0 = Taylor1([U], n)   # initial condition as function of a, b
    v0 = Taylor1([V], n)   # initial condition as function of a, b

    U, V = integ(u0, v0, (i*step)..((i+1)*step), bounds);

    xs, ys = calculate_set(U, V, bounds)
    
    plot!(xs, ys)
end
p

(uΔ, vΔ) = ([-3.20024e-09, 3.20024e-09], [-1.85023e-09, 1.85023e-09])
(uΔ, vΔ) = ([-6.40048e-09, 6.40048e-09], [-3.70046e-09, 3.70046e-09])
(uΔ, vΔ) = ([-1.2801e-08, 1.2801e-08], [-7.40092e-09, 7.40092e-09])
i = 1
(uΔ, vΔ) = ([-1.11629e-09, 1.11629e-09], [-5.74319e-10, 5.74319e-10])
(uΔ, vΔ) = ([-2.23257e-09, 2.23257e-09], [-1.14864e-09, 1.14864e-09])
(uΔ, vΔ) = ([-4.46514e-09, 4.46514e-09], [-2.29728e-09, 2.29728e-09])
i = 2
(uΔ, vΔ) = ([-1.62596e-10, 1.62596e-10], [-9.97749e-11, 9.97749e-11])
(uΔ, vΔ) = ([-3.25191e-10, 3.25191e-10], [-1.9955e-10, 1.9955e-10])
(uΔ, vΔ) = ([-6.50382e-10, 6.50382e-10], [-3.991e-10, 3.991e-10])
i = 3
(uΔ, vΔ) = ([-6.86878e-11, 6.86878e-11], [-3.8993e-11, 3.8993e-11])
(uΔ, vΔ) = ([-1.37376e-10, 1.37376e-10], [-7.79859e-11, 7.79859e-11])
(uΔ, vΔ) = ([-2.74752e-10, 2.74752e-10], [-1.55972e-10, 1.55972e-10])
i = 4
(uΔ, vΔ) = ([-9.59259e-12, 9.59259e-12], [-4.88674e-12, 4.88674e-12])
(uΔ, vΔ) = ([-1.91852e-11, 1.91852e-11], [-9.77347e-12, 9.77347e-12])
(uΔ, v

(uΔ, vΔ) = ([-1.35275e-06, 1.35275e-06], [-6.91355e-07, 6.91355e-07])
i = 52
(uΔ, vΔ) = ([-2.00577e-07, 2.00577e-07], [-1.00636e-07, 1.00636e-07])
(uΔ, vΔ) = ([-4.01154e-07, 4.01154e-07], [-2.01272e-07, 2.01272e-07])
(uΔ, vΔ) = ([-8.02307e-07, 8.02307e-07], [-4.02543e-07, 4.02543e-07])
(uΔ, vΔ) = ([-1.60462e-06, 1.60462e-06], [-8.05085e-07, 8.05085e-07])
(uΔ, vΔ) = ([-3.20923e-06, 3.20923e-06], [-1.61017e-06, 1.61017e-06])
i = 53
(uΔ, vΔ) = ([-1.26398e-07, 1.26398e-07], [-6.52081e-08, 6.52081e-08])
(uΔ, vΔ) = ([-2.52795e-07, 2.52795e-07], [-1.30417e-07, 1.30417e-07])
(uΔ, vΔ) = ([-5.05589e-07, 5.05589e-07], [-2.60833e-07, 2.60833e-07])
(uΔ, vΔ) = ([-1.01118e-06, 1.01118e-06], [-5.21665e-07, 5.21665e-07])
i = 54
(uΔ, vΔ) = ([-4.90427e-08, 4.90427e-08], [-2.59879e-08, 2.59879e-08])
(uΔ, vΔ) = ([-9.80853e-08, 9.80853e-08], [-5.19758e-08, 5.19758e-08])
(uΔ, vΔ) = ([-1.96171e-07, 1.96171e-07], [-1.03952e-07, 1.03952e-07])
i = 55
(uΔ, vΔ) = ([-1.34245e-08, 1.34245e-08], [-6.95366e-09, 6.9536

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip9702)" style="stroke:#ed5d92; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.0862,153.694 44.0535,

In [20]:
t = Taylor1(10)

 1.0 t + 𝒪(t¹¹)

In [21]:
?Taylor1

search: Taylor1 TaylorN taylor_var TaylorModel TaylorModels TaylorSeries



```
Taylor1{T<:Number} <: AbstractSeries{T}
```

DataType for polynomial expansions in one independent variable.

**Fields:**

  * `coeffs :: Array{T,1}` Expansion coefficients; the $i$-th   component is the coefficient of degree $i-1$ of the expansion.
  * `order  :: Int64` Maximum order (degree) of the polynomial.

Note that `Taylor1` variables are callable. For more information, see [`evaluate`](@ref).

```
Taylor1([T::Type=Float64], [order::Int=1])
```

Shortcut to define the independent variable of a `Taylor1{T}` polynomial of given `order`. The default type for `T` is `Float64`.

```julia
julia> Taylor1(16)
 1.0 t + 𝒪(t¹⁷)

julia> Taylor1(Rational{Int}, 4)
 1//1 t + 𝒪(t⁵)
```


In [ ]:
TaylorModel(3, 1..1, 0.5..1.5)